In [1]:
# Import necessary libraries
import pandas as pd

# Load parquet and csv files into DataFrames and print headers
df_p65 = pd.read_parquet('full_p65_with_nearest.parquet')
print("Headers for full_p65_with_nearest.parquet:")
print(df_p65.head())
print(df_p65.columns)

df_jcc = pd.read_csv('jcc_counts.csv')
print("\nHeaders for jcc_counts.csv:")
print(df_jcc.head())
print(df_jcc.columns)

Headers for full_p65_with_nearest.parquet:
  benchmark                        title  \
0         1       Division Director ‐ PX   
1         1       Division Director ‐ PX   
2         1       Division Director ‐ PX   
3         2  Division Operations Manager   
4         2  Division Operations Manager   

                                  group    code  \
0  Executives and Senior Administrators  PA0112   
1  Executives and Senior Administrators  PA0112   
2  Executives and Senior Administrators  PA0112   
3  Executives and Senior Administrators  PA0113   
4  Executives and Senior Administrators  PA0113   

                                        family  BU schedule  original_range  \
0  PA01 ‐ Executives and Senior Administrators  XE      N00            27.0   
1  PA01 ‐ Executives and Senior Administrators  XE      N00            27.0   
2  PA01 ‐ Executives and Senior Administrators  XE      N00            27.0   
3  PA01 ‐ Executives and Senior Administrators  SS      200          

In [2]:
# Group by 'code', find the index of max 'range_change' for each group, and select those rows
df_p65_filtered = df_p65.loc[df_p65.groupby('code')['range_change'].idxmax()]

print(df_p65_filtered.head())
print(df_p65_filtered.columns)

# Inner join df_jcc with df_p65_filtered on 'code' and 'JCC'
df_p65_filtered = pd.merge(df_p65_filtered, df_jcc[['JCC', 'Count']].rename(columns={'Count': 'headcount', 'JCC': 'code'}), on='code', how='left')

df_p65_filtered['headcount'] = df_p65_filtered['headcount'].fillna(0).astype(int)

df_p65_filtered.head()

   benchmark                        title  \
0          1       Division Director ‐ PX   
4          2  Division Operations Manager   
6          3           Office Assistant 1   
9          4                    Secretary   
12         5        Executive Secretary 3   

                                   group    code  \
0   Executives and Senior Administrators  PA0112   
4   Executives and Senior Administrators  PA0113   
6      Administrative and Office Support  PB0102   
9      Administrative and Office Support  PB0130   
12     Administrative and Office Support  PB0137   

                                         family  BU schedule  original_range  \
0   PA01 ‐ Executives and Senior Administrators  XE      N00            27.0   
4   PA01 ‐ Executives and Senior Administrators  SS      200            24.0   
6                 PB01 ‐ General Administration  GP      200             8.0   
9                 PB01 ‐ General Administration  XE      N00            11.0   
12              

,benchmark,title,group,code,family,BU,schedule,original_range,step,ak_annual_salary,market_target_annual_salary,percentile,nearest_range,range_change,headcount
0,1,Division Director ‐ PX,Executives and Senior Administrators,PA0112,PA01 ‐ Executives and Senior Administrators,XE,N00,27.0,A,117371,146594.0,65,30,3,78
1,2,Division Operations Manager,Executives and Senior Administrators,PA0113,PA01 ‐ Executives and Senior Administrators,SS,200,24.0,E,124654,129996.0,65,25,1,61
2,3,Office Assistant 1,Administrative and Office Support,PB0102,PB01 ‐ General Administration,GP,200,8.0,A,34983,43186.0,65,12,4,68
3,4,Secretary,Administrative and Office Support,PB0130,PB01 ‐ General Administration,XE,N00,11.0,A,43115,56890.0,65,15,4,0
4,5,Executive Secretary 3,Administrative and Office Support,PB0137,PB01 ‐ General Administration,XE,N00,16.0,A,59534,73930.0,65,19,3,11


In [3]:
# Get 25 rows with lowest headcount
lowest_headcount = df_p65_filtered.nsmallest(25, 'headcount')[['code', 'title', 'headcount', 'range_change']]
print("Lowest 25 headcounts:")
print(lowest_headcount)

# Get 25 rows with highest headcount
highest_headcount = df_p65_filtered.nlargest(25, 'headcount')[['code', 'title', 'headcount', 'range_change']]
print("\nHighest 25 headcounts:")
print(highest_headcount)

# Get 25 rows with highest range_change
highest_range_change = df_p65_filtered.nlargest(25, 'range_change')[['code', 'title', 'headcount', 'range_change']]
print("\nHighest 25 range changes:")
print(highest_range_change)

Lowest 25 headcounts:
       code                                              title  headcount  \
3    PB0130                                          Secretary          0   
21   PB0325                            Labor Relations Manager          0   
43   PB07A3                            Application Developer 3          0   
44   PB07B2                              IT Business Analyst 2          0   
45   PB07C2                     IT Client Support Specialist 2          0   
46   PB07D2                           Database Administrator 2          0   
47   PB07E1                            IT Support Technician 1          0   
48   PB07J2                               IT Project Manager 2          0   
49   PB07M1                                       IT Manager 1          0   
50   PB07N2        Network and Telecommunications Specialist 2          0   
51   PB07P1                   IT Policy and Planning Analyst 1          0   
52   PB07R1                       IT Architecture Spec

In [4]:
# Total count of job classes studied that have a 0 headcount
zero_headcount_count = (df_p65_filtered['headcount'] == 0).sum()
print(f"Total count of job classes studied with a 0 headcount: {zero_headcount_count}")

# Count of the job titles that need a three or more range increase
high_range_change_count = (df_p65_filtered['range_change'] >= 3).sum()
print(f"Count of job titles that need a three or more range increase: {high_range_change_count}")

# Weighted average range increase needed based on the headcount
weighted_average_range_increase = (df_p65_filtered['headcount'] * df_p65_filtered['range_change']).sum() / df_p65_filtered['headcount'].sum()
print(f"Weighted average range increase needed based on the headcount: {weighted_average_range_increase:.2f}")

Total count of job classes studied with a 0 headcount: 33
Count of job titles that need a three or more range increase: 104
Weighted average range increase needed based on the headcount: 1.49


In [5]:
# Count of the job titles and headcount that need a three or more range increase
high_range_change_count = df_p65_filtered[df_p65_filtered['range_change'] >= 3]
print(f"Count of job titles that need a three or more range increase: {high_range_change_count.shape[0]}")
print(f"Headcount of employees that need a three or more range increase: {high_range_change_count['headcount'].sum()}")

# Percentage of workforce that needs at least a 1 range increase
at_least_one_range_increase = df_p65_filtered[df_p65_filtered['range_change'] > 0]
percentage_at_least_one_range_increase = (at_least_one_range_increase['headcount'].sum() / df_p65_filtered['headcount'].sum()) * 100
print(f"Percentage of workforce that needs at least a 1 range increase: {percentage_at_least_one_range_increase:.2f}%")

# Percentage of workforce that needs at least a 2 range increase
at_least_two_range_increase = df_p65_filtered[df_p65_filtered['range_change'] >= 2]
percentage_at_least_two_range_increase = (at_least_two_range_increase['headcount'].sum() / df_p65_filtered['headcount'].sum()) * 100
print(f"Percentage of workforce that needs at least a 2 range increase: {percentage_at_least_two_range_increase:.2f}%")

# Number of employees that need at least a 3 range increase
at_least_three_range_increase = df_p65_filtered[df_p65_filtered['range_change'] >= 3]
employees_at_least_three_range_increase = at_least_three_range_increase['headcount'].sum()
print(f"Number of employees that need at least a 3 range increase: {employees_at_least_three_range_increase}")

# Number of employees that need at least a 4 range increase
at_least_four_range_increase = df_p65_filtered[df_p65_filtered['range_change'] >= 4]
employees_at_least_four_range_increase = at_least_four_range_increase['headcount'].sum()
print(f"Number of employees that need at least a 4 range increase: {employees_at_least_four_range_increase}")

Count of job titles that need a three or more range increase: 104
Headcount of employees that need a three or more range increase: 1500
Percentage of workforce that needs at least a 1 range increase: 73.21%
Percentage of workforce that needs at least a 2 range increase: 48.84%
Number of employees that need at least a 3 range increase: 1500
Number of employees that need at least a 4 range increase: 505
